# WordCount

In [ ]:
import sys
sys.path.append("..")
from helpers.path_translation import translate_to_file_string
from pyspark.sql import SparkSession
import re

In [ ]:
input_file = translate_to_file_string("../../data/nasa_http_log_1995.csv")

In [ ]:
#create a SparkSession without local master and app name
spark = (SparkSession
       .builder 
       .master("local[*]") 
       .appName("WordCount")
       .getOrCreate())
# read file 
spark.sparkContext.setLogLevel("ERROR")
input = spark.sparkContext.textFile(input_file)
# The regular expression to find html get requests
filter_pattern = re.compile(".*GET,/.*\.html")
match_pattern = re.compile('GET,/.*\.html')
filtered = input.filter(lambda s: filter_pattern.match(s))
pages = filtered.map(lambda s: (match_pattern.search(s).group()[4:], 1))
popularity = pages.reduceByKey(lambda a, b: a+b)

In [ ]:
print(popularity.collect())
#print(filtered.collect())

In [ ]:
spark.stop()